In [182]:
import pandas as pd
import numpy as np

## Standardising Data

In [183]:
# Function for standardising and unstandardising columns
def standardise_columns(df, cols):
    subset_df = df[cols]
    subset_df = 0.8 * ((subset_df - subset_df.min()) / (subset_df.max() - subset_df.min())) + 0.1
    return subset_df

def unstandardise_columns(df, cols, max_val, min_val):
    subset_df = df[cols]
    subset_df = ((subset_df - subset_df.min()) / 0.8) * (max_val - min_val) + min_val
    return subset_df

## Reading Data

In [184]:
my_data = pd.read_excel("River-Data-Lagged.xlsx")
my_data.drop(["Unnamed: 0"], axis=1, inplace=True)
my_data.head(10)

,Date,Skelton MDF (Cumecs),Crakehill MDF (t-1),Skip Bridge MDF (t-1),Westwick MDF (t-1),Skelton MDF (t-1),Crakehill MDF (t-2),Skip Bridge MDF (t-2),Westwick MDF (t-2),Skelton MDF (t-2),...,Malham Tarn DRT (t-1),Snaizeholme DRT (t-1),Arkengarthdale DRT (t-2),East Cowton DRT (t-2),Malham Tarn DRT (t-2),Snaizeholme DRT (t-2),Arkengarthdale DRT (t-3),East Cowton DRT (t-3),Malham Tarn DRT (t-3),Snaizeholme DRT (t-3)
0,1993-01-04,23.47,9.46,4.124,8.057,23.60,9.95,4.239,8.622,24.86,...,0.8,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,4.0
1,1993-01-05,60.70,9.41,4.363,7.925,23.47,9.46,4.124,8.057,23.60,...,0.8,61.6,0.0,0.0,0.8,0.0,0.0,0.0,0.8,0.0
2,1993-01-06,98.01,26.30,11.962,58.704,60.70,9.41,4.363,7.925,23.47,...,33.6,111.2,2.4,24.8,0.8,61.6,0.0,0.0,0.8,0.0
3,1993-01-07,56.99,32.10,10.237,34.416,98.01,26.30,11.962,58.704,60.70,...,1.6,0.8,11.2,5.6,33.6,111.2,2.4,24.8,0.8,61.6
4,1993-01-08,56.66,19.30,7.254,22.263,56.99,32.10,10.237,34.416,98.01,...,17.6,36.0,0.0,0.0,1.6,0.8,11.2,5.6,33.6,111.2
5,1993-01-09,78.10,22.00,7.266,29.587,56.66,19.30,7.254,22.263,56.99,...,1.6,2.4,5.6,4.0,17.6,36.0,0.0,0.0,1.6,0.8
6,1993-01-10,125.70,35.50,8.153,60.253,78.10,22.00,7.266,29.587,56.66,...,55.2,104.8,1.6,0.0,1.6,2.4,5.6,4.0,17.6,36.0
7,1993-01-11,195.90,51.00,13.276,93.951,125.70,35.50,8.153,60.253,78.10,...,76.0,136.8,14.4,0.8,55.2,104.8,1.6,0.0,1.6,2.4
8,1993-01-12,125.40,65.50,25.561,69.503,195.90,51.00,13.276,93.951,125.70,...,12.0,28.0,20.8,2.4,76.0,136.8,14.4,0.8,55.2,104.8
9,1993-01-13,161.50,32.00,20.715,40.514,125.40,65.50,25.561,69.503,195.90,...,0.8,24.0,10.4,16.0,12.0,28.0,20.8,2.4,76.0,136.8


In [185]:
target_cols = ["Skelton MDF (Cumecs)"]
flow_cols = [col for col in my_data.columns if "MDF (t" in col]
rain_cols = [col for col in my_data.columns if "DRT" in col]

In [186]:
feature_cols = flow_cols[:4] + rain_cols[:4]
feature_cols

['Crakehill MDF (t-1)',
 'Skip Bridge MDF (t-1)',
 'Westwick MDF (t-1)',
 'Skelton MDF (t-1)',
 'Arkengarthdale DRT (t-1)',
 'East Cowton DRT (t-1)',
 'Malham Tarn DRT (t-1)',
 'Snaizeholme DRT (t-1)']

In [187]:
training_df = my_data[target_col + features]
training_df.head(10)

,Skelton MDF (Cumecs),Crakehill MDF (t-1),Skip Bridge MDF (t-1),Westwick MDF (t-1),Skelton MDF (t-1),Arkengarthdale DRT (t-1),East Cowton DRT (t-1),Malham Tarn DRT (t-1),Snaizeholme DRT (t-1)
0,23.47,9.46,4.124,8.057,23.60,0.0,0.0,0.8,0.0
1,60.70,9.41,4.363,7.925,23.47,2.4,24.8,0.8,61.6
2,98.01,26.30,11.962,58.704,60.70,11.2,5.6,33.6,111.2
3,56.99,32.10,10.237,34.416,98.01,0.0,0.0,1.6,0.8
4,56.66,19.30,7.254,22.263,56.99,5.6,4.0,17.6,36.0
5,78.10,22.00,7.266,29.587,56.66,1.6,0.0,1.6,2.4
6,125.70,35.50,8.153,60.253,78.10,14.4,0.8,55.2,104.8
7,195.90,51.00,13.276,93.951,125.70,20.8,2.4,76.0,136.8
8,125.40,65.50,25.561,69.503,195.90,10.4,16.0,12.0,28.0
9,161.50,32.00,20.715,40.514,125.40,7.2,4.0,0.8,24.0


In [188]:
std_training_df = standardise_columns(training_df, target_col + features)
std_training_df.head(10)

,Skelton MDF (Cumecs),Crakehill MDF (t-1),Skip Bridge MDF (t-1),Westwick MDF (t-1),Skelton MDF (t-1),Arkengarthdale DRT (t-1),East Cowton DRT (t-1),Malham Tarn DRT (t-1),Snaizeholme DRT (t-1)
0,0.135600,0.127163,0.131519,0.113121,0.135834,0.100000,0.100000,0.102540,0.100000
1,0.202620,0.126980,0.133932,0.112837,0.135600,0.108526,0.219807,0.102540,0.283333
2,0.269783,0.188979,0.210648,0.222008,0.202620,0.139787,0.127053,0.206667,0.430952
3,0.195941,0.210269,0.193233,0.169791,0.269783,0.100000,0.100000,0.105079,0.102381
4,0.195347,0.163283,0.163118,0.143663,0.195941,0.119893,0.119324,0.155873,0.207143
5,0.233942,0.173194,0.163239,0.159409,0.195347,0.105684,0.100000,0.105079,0.107143
6,0.319630,0.222749,0.172194,0.225338,0.233942,0.151155,0.103865,0.275238,0.411905
7,0.446001,0.279646,0.223914,0.297786,0.319630,0.173890,0.111594,0.341270,0.507143
8,0.319090,0.332871,0.347939,0.245225,0.446001,0.136945,0.177295,0.138095,0.183333
9,0.384075,0.209902,0.299016,0.182901,0.319090,0.125577,0.119324,0.102540,0.171429


## Basic ANN Class

In [242]:
from sklearn.metrics import *

In [287]:
class BasicAnn:
    def __init__(self, layers):
        weight_shapes = [(layers[i],layers[i-1]) for i in range(1, len(layers))]
        self.weights = [np.random.standard_normal(s)/s[1]**0.5 for s in weight_shapes]
        self.biases = [np.random.randn(y,1) for y in layers[1:]]
    
    def train(self, features, targets):
        for i, row in features.iterrows():
            print(format_row(row, row.size))
#         preds = features.apply(lambda r: myAnn.forward_pass(format_row(r, r.size)), axis=1).explode().explode()
#         real_vals = targets.iloc[:, 0]
#         error = real_vals - preds
#         return preds, error
    
    def forward_pass(self, data):
        activations = []
        for w, b in zip(self.weights, self.biases):
            activation = self.activation(np.matmul(w, data) + b)
            activations.append(activation)

        return activations
    
    def activation(self, x, func_name="sigmoid"):
        if func_name == "sigmoid":
            return 1/(1+np.exp(-x))
    

In [288]:
# Utility Functions
def format_row(row, row_size):
    return row.to_numpy().reshape(row_size, 1)

In [289]:
myAnn = BasicAnn((8,10,1))

In [290]:
features = std_training_df[feature_cols]
targets = std_training_df[target_cols]
myAnn.train(features, targets)

[[0.12716344]
 [0.13151864]
 [0.11312096]
 [0.1358339 ]
 [0.1       ]
 [0.1       ]
 [0.10253968]
 [0.1       ]]
[[0.1269799 ]
 [0.1339315 ]
 [0.11283717]
 [0.13559988]
 [0.10852575]
 [0.21980676]
 [0.10253968]
 [0.28333333]]
[[0.18897862]
 [0.21064839]
 [0.22200792]
 [0.20261968]
 [0.13978686]
 [0.12705314]
 [0.20666667]
 [0.43095238]]
[[0.21026888]
 [0.19323339]
 [0.16979068]
 [0.26978349]
 [0.1       ]
 [0.1       ]
 [0.10507937]
 [0.10238095]]
[[0.16328347]
 [0.16311804]
 [0.14366271]
 [0.1959411 ]
 [0.11989343]
 [0.11932367]
 [0.15587302]
 [0.20714286]]
[[0.17319446]
 [0.16323919]
 [0.15940872]
 [0.19534705]
 [0.10568384]
 [0.1       ]
 [0.10507937]
 [0.10714286]]
[[0.22274938]
 [0.17219404]
 [0.22533814]
 [0.23394239]
 [0.15115453]
 [0.10386473]
 [0.2752381 ]
 [0.41190476]]
[[0.27964577]
 [0.22391409]
 [0.29778612]
 [0.3196298 ]
 [0.17388988]
 [0.1115942 ]
 [0.34126984]
 [0.50714286]]
[[0.33287143]
 [0.34793922]
 [0.24522489]
 [0.44600073]
 [0.13694494]
 [0.17729469]
 [0.13809524

 [0.1       ]]
[[0.21247132]
 [0.25800712]
 [0.17092369]
 [0.26969348]
 [0.1       ]
 [0.1       ]
 [0.2015873 ]
 [0.11190476]]
[[0.21797742]
 [0.25133389]
 [0.18880241]
 [0.25678636]
 [0.11705151]
 [0.1       ]
 [0.16095238]
 [0.13571429]]
[[0.1988896 ]
 [0.24175311]
 [0.19830506]
 [0.24967575]
 [0.12273535]
 [0.1       ]
 [0.31587302]
 [0.20952381]]
[[0.21724328]
 [0.25147523]
 [0.23670262]
 [0.28416673]
 [0.18241563]
 [0.26618357]
 [0.34634921]
 [0.21666667]]
[[0.32332752]
 [0.26494283]
 [0.20991462]
 [0.34249178]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.20072497]
 [0.20353096]
 [0.15792527]
 [0.24438329]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.10952381]]
[[0.17282738]
 [0.19411171]
 [0.1464791 ]
 [0.20243966]
 [0.1       ]
 [0.1       ]
 [0.19904762]
 [0.12619048]]
[[0.1698908 ]
 [0.20359153]
 [0.15598605]
 [0.20015346]
 [0.11420959]
 [0.1       ]
 [0.13809524]
 [0.15      ]]
[[0.18383959]
 [0.20560057]
 [0.16723227]
 [0.21979316]
 [0.1       ]
 [0.1      

 [0.1       ]]
[[0.10488208]
 [0.10505792]
 [0.10361831]
 [0.10765966]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.10381756]
 [0.10470458]
 [0.10317113]
 [0.10614753]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.10315683]
 [0.10459352]
 [0.10251326]
 [0.10496663]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.10264293]
 [0.10425027]
 [0.10213487]
 [0.10434018]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.10227586]
 [0.10401807]
 [0.10179303]
 [0.10330689]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.1019822 ]
 [0.10387673]
 [0.10151569]
 [0.1035157 ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]]
[[0.10172525]
 [0.10460362]
 [0.1013867 ]
 [0.10275424]
 [0.10852575]
 [0.11545894]
 [0.15079365]
 [0.1       ]]
[[0.10183537]
 [0.10724868]
 [0.1012835 ]
 [0.10282984]
 [0.10284192]
 [0.1115942 ]
 [0.13301587]
 [0.1       ]]
[[0.10392769]
 [0.1055829 ]
 [0.10180593]
 [0.1035175 ]
 [0.17673179]
 [0.2507246

In [253]:
preds

0       0.576405
1       0.580893
2       0.582517
3       0.577123
4       0.578249
          ...   
1443    0.576667
1444     0.57729
1445    0.577027
1446    0.576766
1447    0.576647
Length: 1448, dtype: object

In [254]:
error

0      -0.440805
1      -0.378273
2      -0.312733
3      -0.381182
4      -0.382902
          ...   
1443   -0.430176
1444   -0.432329
1445   -0.430914
1446   -0.421706
1447   -0.412334
Length: 1448, dtype: object